# [Database Secret Engine](https://developer.hashicorp.com/vault/tutorials/db-credentials/database-secrets)

In [15]:
%env WORKDIR=/tmp/vault            

env: WORKDIR=/tmp/vault


In [16]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

## Install PostgreSQL

In [17]:
%%bash

export DOCKER_CONFIG=$HOME/.config/containers

# Add Helm repository by Bitnami
helm repo add bitnami https://charts.bitnami.com/bitnami

# Update Helm index charts
helm repo update
kubectl apply -f local-pv.yaml
kubectl apply -f pv-claim.yaml

helm install postgresql-dev -f postgres.yaml oci://registry-1.docker.io/bitnamicharts/postgresql -n vault --set volumePermissions.enabled=true

"bitnami" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "secrets-store-csi-driver" chart repository
...Successfully got an update from the "vault" chart repository
...Successfully got an update from the "hashicorp" chart repository
...Successfully got an update from the "postgres-operator-charts" chart repository
...Successfully got an update from the "jaeger-all-in-one" chart repository
...Successfully got an update from the "jaegertracing" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "minio" chart repository
...Successfully got an update from the "datadog" chart repository
...Successfully got an update from the "signoz" chart repository
...Successfully got an update from the "external-secrets" chart repository
...Successfully got an update from the "gitlab" chart repository
...Successfully got a

Pulled: registry-1.docker.io/bitnamicharts/postgresql:16.6.6
Digest: sha256:a8a0fd5ecbec861cc8462a417a8804c182caa2ee1666abc1a0f8a7f9126c2e40


NAME: postgresql-dev
LAST DEPLOYED: Tue May  6 16:27:42 2025
NAMESPACE: vault
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
CHART NAME: postgresql
CHART VERSION: 16.6.6
APP VERSION: 17.4.0

Did you know there are enterprise versions of the Bitnami catalog? For enhanced secure software supply chain features, unlimited pulls from Docker, LTS support, or application customization, see Bitnami Premium or Tanzu Application Catalog. See https://www.arrow.com/globalecs/na/vendors/bitnami for more information.

** Please be patient while the chart is being deployed **

PostgreSQL can be accessed via port 5432 on the following DNS names from within your cluster:

    postgresql-dev.vault.svc.cluster.local - Read/Write connection

To get the password for "postgres" run:

    export POSTGRES_ADMIN_PASSWORD=$(kubectl get secret --namespace vault postgresql-dev -o jsonpath="{.data.postgres-password}" | base64 -d)

To get the password for "app1" run:

    export POSTGRES_PASSWORD=$(kubectl ge

In [ ]:
# ! kubectl get events -n vault
! kubectl get pods -n vault

LAST SEEN   TYPE      REASON                  OBJECT                                        MESSAGE
89m         Normal    Provisioning            persistentvolumeclaim/data-vault-0            External provisioner is provisioning volume for claim "vault/data-vault-0"
89m         Normal    ExternalProvisioning    persistentvolumeclaim/data-vault-0            Waiting for a volume to be created either by the external provisioner 'k8s.io/minikube-hostpath' or manually by the system administrator. If volume creation is delayed, please verify that the provisioner is running and correctly registered.
89m         Normal    ProvisioningSucceeded   persistentvolumeclaim/data-vault-0            Successfully provisioned volume pvc-ee2f933d-684d-4945-80a2-77dc5071ea90
89m         Normal    ExternalProvisioning    persistentvolumeclaim/data-vault-1            Waiting for a volume to be created either by the external provisioner 'k8s.io/minikube-hostpath' or manually by the system administrator. If vo

In [ ]:
! kubectl get pods -n vault

Create role that will be used by Vault

In [19]:
! kubectl exec -it postgresql-dev-0 -n vault -- sh -c 'export PGPASSWORD=StrongPassword; psql --host 127.0.0.1 -U postgres -c "CREATE ROLE \"ro\" NOINHERIT;"'
! kubectl exec -it postgresql-dev-0 -n vault -- sh -c 'export PGPASSWORD=StrongPassword; psql --host 127.0.0.1 -U postgres -c "GRANT SELECT ON ALL TABLES IN SCHEMA public TO \"ro\";"'

Defaulted container "postgresql" out of: postgresql, init-chmod-data (init)
psql: error: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
command terminated with exit code 2
Defaulted container "postgresql" out of: postgresql, init-chmod-data (init)
ERROR:  role "ro" does not exist
command terminated with exit code 1


## Enable the database secret engine

In [8]:
%%bash
export POSTGRES_URL=postgresql-dev.vault.svc.cluster.local
# Enable engine
vault secrets enable database

# Configure the database secrets engine with the connection credentials for the Postgres database.
vault write database/config/postgresql \
     plugin_name=postgresql-database-plugin \
     connection_url="postgresql://{{username}}:{{password}}@$POSTGRES_URL/postgres?sslmode=disable" \
     allowed_roles=* \
     username="postgres" \
     password="StrongPassword"

Error enabling: Error making API request.

URL: POST https://127.0.0.1:8200/v1/sys/mounts/database
Code: 400. Errors:

* path is already in use at database/


Success! Data written to: database/config/postgresql


## Create a role in Vault

In [9]:
%%bash
cat > ${WORKDIR}/readonly.sql <<EOF
CREATE ROLE "{{name}}" WITH LOGIN PASSWORD '{{password}}' VALID UNTIL '{{expiration}}' INHERIT;
GRANT ro TO "{{name}}";
EOF

In [10]:
%%bash
vault write database/roles/readonly \
      db_name=postgresql \
      creation_statements=@${WORKDIR}/readonly.sql \
      default_ttl=60s \
      max_ttl=120s

Success! Data written to: database/roles/readonly


## Request Credentials

In [11]:
! vault read -format=json database/creds/readonly 

{
  "request_id": "e9f32915-aca6-a4a2-f9d9-868ec785e637",
  "lease_id": "database/creds/readonly/TOCbS8aXq1kCAf05QbRv1uNf",
  "lease_duration": 60,
  "renewable": true,
  "data": {
    "password": "Xq-XiHxv8yIHLKiqNusV",
    "username": "v-root-readonly-7juPGr2ENeZLYxnbsR5E-1746539284"
  },
  "warnings": null
}


In [12]:
! vault lease revoke -prefix database/creds/readonly

All revocation operations queued successfully!


## Rotate Root password

In [13]:
! vault write -force database/rotate-root/postgresql

Success! Data written to: database/rotate-root/postgresql


## Clean up

In [ ]:
# Run on to clean up
# Uncomment to run the cleanup
#%%bash
vault secrets disable database
helm uninstall postgresql-dev -n vault
kubectl delete pvc postgresql-data-claim -n vault
kubectl delete pv postgresql-data -n vault

Success! Disabled the secrets engine (if it existed) at: database/
release "postgresql-dev" uninstalled
persistentvolumeclaim "postgresql-data-claim" deleted


persistentvolume "postgresql-data" deleted
